In [1]:
!pip install -q tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00


In [2]:
import math
from dataclasses import dataclass
import inspect    #helps with inspecting properties within an object
import torch
import torch.nn as nn
from torch.nn import functional as F

In [3]:
# device agnostic code
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, "mps"):
    device = "mps"

print(f"Using device: {device}")

Using device: cuda


In [4]:
class CausalSelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        # regularization
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        # not really a 'bias', more of a mask, but following the OpenAI/HF naming though
        self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                     .view(1, 1, config.block_size, config.block_size))
        self.c_proj.NANOGPT_WEIGHT_INIT = 1.0

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)
        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        # nh is "number of heads", hs is "head size", and C (number of channels) = nh * hs
        # e.g. in GPT-2 (124M), n_head=12, hs=64, so nh*hs=C=768 channels in the Transformer
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # # Normal attention (materializes the large (T,T) matrix for all the queries and keys)
        # att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        # att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
        # att = F.softmax(att, dim=-1)
        # y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)

        """Switching to Flash Attention"""
        y = F.scaled_dot_product_attention(q, k, v, is_causal=True)
        # continuation same for causalattention
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side
        # output projection
        y = self.c_proj(y)
        return y

class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu    = nn.GELU(approximate='tanh')
        self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd)
        self.NANOGPT_WEIGHT_INIT = 1.0     #associated with the projection

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x

class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

@dataclass
class GPTConfig:
    block_size: int = 1024 # max sequence length
    vocab_size: int = 50257 # number of tokens: 50,000 BPE merges + 256 bytes tokens + 1 <|endoftext|> token
    n_layer: int = 12 # number of layers
    n_head: int = 12 # number of heads
    n_embd: int = 768 # embedding dimension

class GPT(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        self.transformer.wte.weight = self.lm_head.weight  #weight sharing scheme linear.weight.shape(vocab_size, n_embd)

        # apply this at initialization. we could have set this while defining each layer
        self.apply(self._init_weights) #applies a function to an nn.Module object

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
          sd = 0.02
          if hasattr(module, "NANOGPT_WEIGHT_INIT"):
            sd *= (2 * self.config.n_layer)**-0.5
          torch.nn.init.normal_(module.weight, mean=0.0, std=sd)
          if module.bias is not None:
            torch.nn.init.zeros_(module.bias)         #initializing bias. there is 'torch.nn.init.ones_'
        elif isinstance(module, nn.Embedding):
          torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)   #intializing weights

    def forward(self, idx, targets=None):
      #input shape is B, T
      B, T = idx.size()
      assert T <= self.config.block_size, f"cannot forward sequence of length {T} because the block size is only {self.config.block_size}"
      #position and token embedding
      pos = torch.arange(0, T, dtype=torch.long, device=idx.device) #input would already have been sent to device
      pos_emb = self.transformer.wpe(pos) # (T, n_embd)  #module dict can be accessed using '.'
      tok_emb = self.transformer.wte(idx) # (B, T, n_embd)
      x = tok_emb + pos_emb
      #blocks
      for block in self.transformer.h:
        x = block(x)
      #LayerNorm
      x = self.transformer.ln_f(x)

      logits = self.lm_head(x)  #B, T, vocab_size

      loss = None
      if targets is not None:
        loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))  # logits should be 2d, targets should be 1d  (B*T, vocab_size, B*T)
      return logits, loss



    @classmethod
    def from_pretrained(cls, model_type):
        """Loads pretrained GPT-2 model weights from huggingface"""
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
            'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
            'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
            'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
        }[model_type]
        config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
        config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints
        # create a from-scratch initialized minGPT model
        config = GPTConfig(**config_args)
        model = GPT(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model

    #configure optimizer
    def configure_optimizers(self, weight_decay, learning_rate, device):
        # start with all of the candidate parameters (that require grad)
        param_dict = {pn: p for pn, p in self.named_parameters()}
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
        # create optim groups. Any parameters that is 2D will be weight decayed, otherwise no.
        # i.e. all weight tensors in matmuls + embeddings decay, all biases and layernorms don't.
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters")
        print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters")
        # Create AdamW optimizer and use the fused version if it is available
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and 'cuda' in device
        print(f"using fused AdamW: {use_fused}")
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=(0.9, 0.95), eps=1e-8, fused=use_fused)
        return optimizer

In [5]:
import tiktoken
import time
# get a static batch of data from shakespare.txt
# enc = tiktoken.get_encoding('gpt2')
# with open('shakespare.txt', 'r', encoding='utf-8') as f:
#   text = f.read()

# text = [:1000]
# tokens = enc.encode(text)
# B, T = 4, 32
# buf = torch.tensor(tokens[:B*T+1])
# buf = buf.to(device)
# x = buf[:-1].view(B, T)
# y = buf[1:].view(B,T)

"""creating an iterative dataloader class"""
class DataLoaderLite:
    def __init__(self, B, T):
        self.B = B
        self.T = T

        # at init load tokens from disk and store them in memory
        with open('shakespare.txt', 'r') as f:
            text = f.read()
        enc = tiktoken.get_encoding('gpt2')
        tokens = enc.encode(text)
        self.tokens = torch.tensor(tokens)
        print(f"loaded {len(self.tokens)} tokens")
        print(f"1 epoch = {len(self.tokens) // (B * T)} batches")

        # state
        self.current_position = 0

    def next_batch(self):
        B, T = self.B, self.T
        buf = self.tokens[self.current_position : self.current_position+B*T+1]
        x = (buf[:-1]).view(B, T) # inputs
        y = (buf[1:]).view(B, T) # targets
        # advance the position in the tensor
        # self.curr_pos += (B*T)+(B*T*torch.randint(-3, 3, (1,)).item()) #randomly assigns batches. add condition for negative positions.
        self.current_position += B * T
        # if loading the next batch would be out of bounds, reset
        if self.current_position + (B * T + 1) > len(self.tokens):
            self.current_position = 0
        return x, y




In [ ]:

torch.manual_seed(42)
if torch.cuda.is_available():
  torch.cuda.manual_seed(42)
#TF32 matmuls
torch.set_float32_matmul_precision('high')  # TF32 computation, 'medium' is bf16

# #without gradient accumulation
# train_data = DataLoaderLite(16, 1024)

#with gradient accumulation
total_batch_size = 524288    #A nice number, close to the OpenAI batch size
B = 4
T = 1024
assert total_batch_size % (B*T) == 0  #confirm divisibility for grad accumulation
grad_accum_steps = total_batch_size // (B*T)
print(f"desired batch size according to OpenAI: {total_batch_size}, grad accumulation steps: {grad_accum_steps}")
train_loader = DataLoaderLite(B, T)


# model = GPT(GPTConfig())
model = GPT(GPTConfig(vocab_size=50304))  #increasing the trainable vocab_size, using nice number for even computation
model.to(device)

#add torch.compile
model = torch.compile(model)

#adding a learning rate scheduler (OpenAI uses a cosine lr schedule)
max_lr = 6e-4
min_lr = max_lr * 0.1
warmup_steps = 10
max_steps = 50

def get_lr(it):
  if it < warmup_steps:
    return max_lr * (it+1) /warmup_steps
  if it > max_steps:
    return min_lr

  decay_ratio = (it - warmup_steps) / (max_steps - warmup_steps)
  assert 0 <= decay_ratio <= 1
  coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))  # start at one and decay to zero

  return min_lr + coeff * (max_lr - min_lr)


# optimizer
# optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
# optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, betas=(0.9, 0.95), eps=1e-8)    #OpenAI settings
optimizer = model.configure_optimizers(weight_decay=0.1, learning_rate=6e-4, device=device)

#training loop
for step in range(max_steps):
  t0 = time.time()
  optimizer.zero_grad()  #never forget to zero the gradient after each step

  # #without gradient accumulation
  # x, y = train_data.next_batch()
  # x, y = x.to(device), y.to(device)
  # #cast loss and logits operation to bfloat16
  # with torch.autocast(device_type=device, dtype=torch.bfloat16):
  #     logits, loss = model(x, y)
  # loss.backward()

  #with gradient accumulation
  loss_accum = 0.0
  for _ in range(grad_accum_steps):
    x, y = train_loader.next_batch()
    x, y = x.to(device), y.to(device)
    #cast loss and logits operation to bfloat16

    """Bfloat16 is not suported by T4 GPU"""
    # with torch.autocast(device_type=device, dtype=torch.bfloat16):
    #   logits, loss = model(x, y)  #a lot aof changes can be made

    logits, loss = model(x, y)
    loss = loss / grad_accum_steps  #this is relevant otherwise the actual value would equal our value divided by accum_no. check with small example
    loss_accum += loss.detach()  #detach instead of item because detach preserves value as tensor (its possible to += a tensor to a scalar value)
    loss.backward()

  # clip gradient to not exceed 1
  norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

  #include lr from scheduler
  lr = get_lr(step)
  for param_group in optimizer.param_groups:  #optimizer object has an attribute called param_groups
    param_group['lr'] = lr

  optimizer.step()
  torch.cuda.synchronize()  #synchronize the cpu and gpu times.
  t1 = time.time()
  dt = t1 - t0

  # #without grad accum
  # tokens_processed = train_loader.B * train_loader.T

  #with grad accum
  tokens_processed = grad_accum_steps * train_loader.B * train_loader.T

  tokens_per_sec = tokens_processed / dt

  print(f"step{step:4d} | loss: {loss_accum.item():.6f} | {lr:.4e} | norm: {norm} | dt:{dt*1000:.2f}ms | tok/sec: {tokens_per_sec}")

desired batch size according to OpenAI: 524288, grad accumulation steps: 128
loaded 338025 tokens
1 epoch = 82 batches
num decayed parameter tensors: 50, with 124,354,560 parameters
num non-decayed parameter tensors: 98, with 121,344 parameters
using fused AdamW: True
step   0 | loss: 10.977739 | 6.0000e-05 | norm: 7.347934722900391 | dt:178321.68ms | tok/sec: 2940.1248496601306
step   1 | loss: 9.790614 | 1.2000e-04 | norm: 4.912006378173828 | dt:134626.98ms | tok/sec: 3894.375473821962
step   2 | loss: 9.363747 | 1.8000e-04 | norm: 8.77891731262207 | dt:134536.00ms | tok/sec: 3897.0090269653087
step   3 | loss: 9.325029 | 2.4000e-04 | norm: 4.76426362991333 | dt:134440.58ms | tok/sec: 3899.774803690933
step   4 | loss: 8.904864 | 3.0000e-04 | norm: 3.6641361713409424 | dt:134430.49ms | tok/sec: 3900.067479236453
step   5 | loss: 8.545564 | 3.6000e-04 | norm: 2.050356388092041 | dt:134138.61ms | tok/sec: 3908.5539357986936
step   6 | loss: 8.289697 | 4.2000e-04 | norm: 2.5339829921722

In [ ]:
#Evaluation code
model.eval()
num_return_sequences=5
max_length = 30

enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode("Hello, I'm a language model") # (8,)
tokens = torch.tensor(tokens, dtype=torch.long)
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)  #(5, 8)  B, T
x = tokens.to(device)

#generate
torch.manual_seed(42)
torch.cuda.manual_seed(42)
while x.size(1) < max_length:
  # forward pass
  with torch.no_grad():
    logits = model(x)  #B, T, vocab_size
    logits = logits[:, -1, :]    #B, vocab_size

    probs = F.softmax(logits, dim=1)  #we always do softmax on the last dim

    # top-K  changes from (5, vocab_size) to (5, 50)
    topk_probs, top_indices = torch.topk(probs, 50, dim=-1)

    ix = torch.multinomial(topk_probs, 1)  #B, 1
    #because our topk_prob is a sample, we'll have to retrieve its indicies
    xcol = torch.gather(top_indices, -1, ix)   #B, 1    index ix from top_indices and gather it in the column
    #append to sequence
    x = torch.cat((x, xcol), dim=1)


# print generated text
for i in range(num_return_sequences):
  tokens = x[i, :max_length].tolist()    #can't use decode on tensor
  decoded = enc.decode(tokens)
  print("Gen", decoded)





loading weights from pretrained gpt: gpt2
Gen Hello, I'm a language modeler. In fact I use that name from C# to describe our development and development workflow. We have many different tools for this.

The best thing about these tools is that they are designed from a pure, easy to use and easy to understand point of view.

A lot of developers feel that having a full-stack application comes down to "let's be great and focus on our own business" rather than "let's focus on all the
Gen Hello, I'm a language model. Language models are not that complicated unless you know to change the behavior of some of your functions, of the variable name, of the variable body and I mean all of them. So I said, "Let's look at our model for variable named "Foo". Well I don't have a variable named Foo, just I have a class. The constructor is created with this 'Foo to Foo' operator:

function foo(expression) { var
Gen Hello, I'm a language model. I make the syntax based on it. For example, it can be used t

tensor(5.)
